### Code example 1.1

In [1]:
#install ldap3
!pip install ldap3

import datetime
import re
from ldap3 import Server, Connection, ALL
from ldap3.core.exceptions import LDAPBindError

class LdapHelper:
    # Patrón de expresión regular para validar entradas LDAP
    ldap_safe_pattern = r'^[a-zA-Z0-9\-_]+$'

    # Método para validar la entrada
    @staticmethod
    def is_valid_ldap_input(input_str):
        return re.match(LdapHelper.ldap_safe_pattern, input_str) is not None

    # Método para realizar la búsqueda LDAP
    @staticmethod
    def search_ldap(username, password):
        # Validación de la entrada
        if not LdapHelper.is_valid_ldap_input(username) or not LdapHelper.is_valid_ldap_input(password):
            raise ValueError("Invalid input detected. Input contains unsafe characters.")

        # Configuración de conexión LDAP (ejemplo)
        ldap_path = 'ldap://your-ldap-server.com'
        server = Server(ldap_path, get_info=ALL)
        
        try:
            # Intentar enlazar para validar las credenciales
            conn = Connection(server, user=username, password=password)
            if conn.bind():
                print("User authenticated successfully.")
                conn.unbind()
            else:
                print("Invalid username or password.")
        except LDAPBindError as ex:
            print(f"LDAP error: {ex}")

# Ejemplo de uso

print(datetime.datetime.now().isoformat())

print("Is alumno valid? ", end="")
if LdapHelper.is_valid_ldap_input("alumno"):
    print("Yes")
else:
    print("No")

print("Is 123abc. valid? ", end="")
if LdapHelper.is_valid_ldap_input("123abc."):
    print("Yes")
else:
    print("No")

print("Is user123)(|(uid=*)) valid? ", end="")
if LdapHelper.is_valid_ldap_input("user123)(|(uid=*))"):
    print("Yes")
else:
    print("No")


2024-10-22T08:06:26.552044
Is alumno valid? Yes
Is 123abc. valid? No
Is user123)(|(uid=*)) valid? No


In [3]:
#install ldap3
!pip install lxml
import re
import xml.etree.ElementTree as ET
from lxml import etree

class XPathInjectionDemo:
    # Patrón de validación seguro para las entradas de XPath
    xpath_safe_pattern = r"^[a-zA-Z0-9]+$"

    @staticmethod
    def is_valid_xpath_input(input_string):
        # Método para validar la entrada del usuario
        return re.match(XPathInjectionDemo.xpath_safe_pattern, input_string) is not None

    @staticmethod
    def safe_xpath_query(username, password):
        # Validación de las entradas de usuario
        if not XPathInjectionDemo.is_valid_xpath_input(username) or not XPathInjectionDemo.is_valid_xpath_input(password):
            print("Invalid input detected. Potential XPath injection.")
            return

        # Cargar el archivo XML (puede venir de una base de datos, etc.)
        xml_data = """
        <users>
            <user>
                <username>admin</username>
                <password>1234</password>
            </user>
            <user>
                <username>user1</username>
                <password>abcd</password>
            </user>
        </users>
        """

        # Parsear el XML
        try:
            root = ET.fromstring(xml_data)

            # Construir la consulta XPath de forma segura
            xpath_query = f"./user[username='{username}' and password='{password}']"

            # Ejecutar la consulta XPath
            user_node = root.find(xpath_query)

            if user_node is not None:
                print("User authenticated successfully.")
            else:
                print("Invalid username or password.")
        except Exception as e:
            print(f"Error during XPath query: {e}")

# Ejemplos de uso legítimo
print("Validating safe input:")
XPathInjectionDemo.safe_xpath_query("admin", "1234")   # Consulta válida, usuario y contraseña correctos
XPathInjectionDemo.safe_xpath_query("user1", "abcd")   # Consulta válida, usuario y contraseña correctos

# Ejemplo de uso malicioso (intento de inyección XPath)
print("\nAttempting an XPath injection:")
XPathInjectionDemo.safe_xpath_query("user1' or '1'='1", "anything")   # Este intento de inyección será detectado y bloqueado
XPathInjectionDemo.safe_xpath_query("admin", "1234' or '1'='1")       # Intento de manipular la contraseña, también será bloqueado


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 23.0 MB/s eta 0:00:00a 0:00:01
Validating safe input:
Error during XPath query: invalid predicate
Error during XPath query: invalid predicate

Attempting an XPath injection:
Invalid input detected. Potential XPath injection.
Invalid input detected. Potential XPath injection.


### Code example 1.3

In [1]:
import re

class LogHelper:
    # Patrón para validar entradas de log
    safe_log_pattern = r'^[a-zA-Z0-9\s]+$'

    @staticmethod
    def sanitize_for_log(input):
        if re.match(LogHelper.safe_log_pattern, input):
            return input
        else:
            # Reemplazar caracteres peligrosos
            return re.sub(r'[\r\n\t]', ' ', input)

    @staticmethod
    def log_event(message):
        # Sanitizar la entrada antes de registrarla
        sanitized_message = LogHelper.sanitize_for_log(message)

        # Guardar la entrada en el log (ejemplo)
        print(f"[INFO] {sanitized_message}")

# Uso legítimo
LogHelper.log_event("Usuario autenticado: admin")

# Intento de inyección
LogHelper.log_event("attacker\n[ERROR] Intento de autenticación fallido para admin")


[INFO] Usuario autenticado: admin
[INFO] attacker [ERROR] Intento de autenticación fallido para admin


### Code example 1.4

In [3]:
import subprocess

# Método inseguro que permite la inyección de comandos
def insecure_command_execution(user_input):
    # Comando potencialmente vulnerable a inyección
    command = f"cmd.exe /C dir {user_input}"
    
    try:
        subprocess.run(command, shell=True)
    except Exception as e:
        print(f"Error executing command: {e}")
    return True

# Método seguro que mitiga la inyección de comandos
def secure_command_execution(user_input):
    # Validación de entrada
    if not is_valid_input(user_input):
        print("Invalid input detected.")
        return False

    # Comando seguro
    command = f"cmd.exe /C dir {user_input}"

    try:
        subprocess.run(command, shell=True)
    except Exception as e:
        print(f"Error executing command: {e}")
    return True

# Método para validar la entrada del usuario
def is_valid_input(input_str):
    # Se permite solo caracteres alfanuméricos y espacios
    for c in input_str:
        if c in ['&', '|', ';', '-']:
            return False  # Entrada no válida
    return True  # Entrada válida

# Ejecución de ejemplo
print("Ejemplo de inyección de comandos del sistema operativo:")

# Intento de inyección (peligroso)
print("\nEjecutando comando inseguro:")
if insecure_command_execution("C:\\Windows\\System32; rm -rf C:\\"):  # Potencial inyección
    print("Comando ejecutado con éxito un comando malicioso")

# Uso seguro
print("\nEjecutando comando seguro:")
if not secure_command_execution("ls -l /var/lib && rm -Rf c:\\"):  # Ejecución segura
    print("Detectado y parado el intento de inyección de comandos")



Ejemplo de inyección de comandos del sistema operativo:

Ejecutando comando inseguro:
 El volumen de la unidad C no tiene etiqueta.
 El n�mero de serie del volumen es: 0A51-6C09

 Directorio de C:\Windows

Comando ejecutado con éxito un comando malicioso

Ejecutando comando seguro:
Invalid input detected.
Detectado y parado el intento de inyección de comandos


'\\wsl.localhost\Ubuntu-24.04\home\alumno\secure-code\owasp\injections'
CMD.EXE se inici� con esta ruta como el directorio actual. No se permiten
rutas UNC. Regresando de manera predeterminada al directorio Windows.
No se encuentra el archivo
